# Wildfire Risk - Data Pre-Prep
__Team 3 - Dave Friesen, John Chen, and Kyle Dalope__<br>
__ADS-508-02-SP23__<br><br>
__GitHub link: https://github.com/davefriesen/wildfire-risk__

In [2]:
__authors__ = ['Dave Friesen', 'John Chen', 'Kyle Dalope']
__contact__ = ['dfriesen@sandiego.edu', 'johnchen@sandiego.edu', 'kdalope@sandiego.edu']
__date__ = '2023-03-20'
__license__ = 'MIT'
__version__ = '1.0.1'

# Setup

In [6]:
# Import basic and data access libraries
import pandas as pd
from profiler import profile, profile_cat

# Import utility libraries
import requests
from bs4 import BeautifulSoup

In [7]:
#clone Git Repo to local machine to run the codes below
!git clone https://github.com/davefriesen/wildfire-risk.git


Cloning into 'wildfire-risk'...
remote: Enumerating objects: 114, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 114 (delta 1), reused 0 (delta 0), pack-reused 104
Receiving objects: 100% (114/114), 8.91 MiB | 13.46 MiB/s, done.
Resolving deltas: 100% (57/57), done.


In [10]:
#check datasets are located within data subdirectory
!ls wildfire-risk/data

Wildland_Fire_Incident_Locations.xlsx  fires.csv  weather.csv


# Data Load and Validation

## Fire Data - Filter for Select Characteristics across Western States

In [11]:
# Create dataframe from select (relevant) columns
cols = ['ContainmentDateTime',
        'ControlDateTime',
        'DiscoveryAcres',
        'EstimatedCostToDate',
        'FinalAcres',
        'FireBehaviorGeneral',
        'FireBehaviorGeneral1',
        'FireBehaviorGeneral2',
        'FireBehaviorGeneral3',
        'FireCause',
        'FireCauseGeneral',
        'FireCauseSpecific',
        'FireDiscoveryDateTime',
        'FireOutDateTime',
        'GACC',
        'IncidentName',
        'IncidentShortDescription',
        'InitialLatitude',
        'InitialLongitude',
        'IsFireCauseInvestigated',
        'IsTrespass',
        'POOCity',
        'POOState',
        'PredominantFuelModel',
        'PrimaryFuelModel']
wfil_df = pd.read_csv('wildfire-risk/data/fires.csv',
                      usecols=cols, low_memory=False)

In [12]:
# Profile selected dataframe
profile(wfil_df)

,Dtype,count,unique,na,na%,mean,std,min,max,skew(>=3),<v0.01,VIF(>=10),examples
ContainmentDateTime,object,35315,33108,53847,60.4,,,,,,,,nan__2022/08/14 21:5
ControlDateTime,object,32570,30401,56592,63.5,,,,,,,,nan__2022/08/18 20:5
DiscoveryAcres,float64,72195,348,16967,19.0,5.8,500.0,,115997.0,191.8,,,0.01__0.01__nan__nan
EstimatedCostToDate,float64,2128,1040,87034,97.6,8193477.0,34440012.7,,800000000.0,13.1,,,nan__nan__nan__nan__
FinalAcres,float64,2631,345,86531,97.0,14.9,318.1,,13440.0,34.3,,,nan__1.23__nan__nan_
FireBehaviorGeneral,object,1498,4,87664,98.3,,,,,,,,nan__nan__nan__nan__
FireBehaviorGeneral1,object,1223,16,87939,98.6,,,,,,,,nan__nan__nan__nan__
FireBehaviorGeneral2,object,1164,16,87998,98.7,,,,,,,,nan__nan__nan__nan__
FireBehaviorGeneral3,object,910,16,88252,99.0,,,,,,,,nan__nan__nan__nan__
FireCause,object,77967,4,11195,12.6,,,,,,,,Undetermined__Undete


In [13]:
# Profile states for further selection
profile_cat(wfil_df, ['POOState'])


POOState - 
US-CA 68.054777
US-OR 17.786725
US-WA 14.158498


In [14]:
# Subset dataframe for select states
wfil_s_df = wfil_df[wfil_df['POOState'].isin(['US-CA', 'US-OR', 'US-WA'])]
profile(wfil_s_df)
profile_cat(wfil_s_df, ['POOState'])

,Dtype,count,unique,na,na%,mean,std,min,max,skew(>=3),<v0.01,VIF(>=10),examples
ContainmentDateTime,object,35315,33108,53847,60.4,,,,,,,,2018/08/05 03:08:00+
ControlDateTime,object,32570,30401,56592,63.5,,,,,,,,2018/08/05 03:08:00+
DiscoveryAcres,float64,72195,348,16967,19.0,5.8,500.0,,115997.0,191.8,,,0.1__0.01__0.1__0.1_
EstimatedCostToDate,float64,2128,1040,87034,97.6,8193477.0,34440012.7,,800000000.0,13.1,,,nan__nan__nan__nan__
FinalAcres,float64,2631,345,86531,97.0,14.9,318.1,,13440.0,34.3,,,nan__nan__nan__nan__
FireBehaviorGeneral,object,1498,4,87664,98.3,,,,,,,,nan__nan__nan__nan__
FireBehaviorGeneral1,object,1223,16,87939,98.6,,,,,,,,nan__nan__nan__nan__
FireBehaviorGeneral2,object,1164,16,87998,98.7,,,,,,,,nan__nan__nan__nan__
FireBehaviorGeneral3,object,910,16,88252,99.0,,,,,,,,nan__nan__nan__nan__
FireCause,object,77967,4,11195,12.6,,,,,,,,Human__Undetermined_



POOState - 
US-CA 68.054777
US-OR 17.786725
US-WA 14.158498


In [15]:
# Write filtered dataframe
wfil_s_df.to_csv('../data/fires.csv', index=False)

## Weather Data - Filter For and Combine Western States

In [16]:
# Not used - station retrieval
#colw = [11, 9, 10, 7, 3, 32, 3, 10]
#coln = ['ID', 'b', 'c', 'd', 'State', 'f', 'g', 'h']
#wstn_df = pd.read_fwf('../data/ghcnd-stations.txt', header=None, widths=colw, names=coln)
#profile_cat(wstn_df, ['State'])
#
#wstn_s_df = wstn_df[wstn_df['State'].isin(['CA', 'OR', 'WA'])]
#profile(wstn_s_df)
#profile_cat(wstn_s_df, ['State'])
#wstn_s_df

In [17]:
# Set base URL for NOAA weather files
url = 'https://www.ncei.noaa.gov/data/global-summary-of-the-month/access/'

# Fetch HTML content of the web page - all file nbames
response = requests.get(url)
response.raise_for_status()

# Parse HTML content
soup = BeautifulSoup(response.text, 'html.parser')

HTTPError: 502 Server Error: Proxy Error for url: https://www.ncei.noaa.gov/data/global-summary-of-the-month/access/

In [ ]:
# Extract file names from the anchor tags in the HTML FOR SELECT STATES
file_names = [fn['href'] \
              for fn in soup.find_all('a')
                  if fn['href'].startswith(('USR0000C', 'USR0000O', 'USR0000W'))]

In [ ]:
# Iterate select files and create combined dataframe
tfn = len(file_names)
print(f'Retrieving {tfn:d} files: ', end='')
n = 1
df = pd.DataFrame()
for f in file_names:
    print('.', end='')
    if n % 10 == 0: print(n, end='')
    df = pd.concat([df, pd.read_csv(url + f)])
    n = n + 1

# Write combined dataframe        
df.to_csv('../data/weather.csv', index=False)
profile(df)

# Release Resources

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [ ]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}